##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Convolutional Neural Network (CNN)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Этот туториал демонстрирует использование простой [Convolutional Neural Network](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) для классификации [CIFAR images](https://www.cs.toronto.edu/~kriz/cifar.html). Так как в этом уроке используется [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview), создание и тренировка модели уложатся в несколько строк кода.


### Импорт TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Загрузка и подготовка датасета CIFAR10


Датасет CIFAR10 содержит 60,000 цветных изображений в 10 классах, по 6,000 в каждом классе. Датасет разделен на  50,000 изображений для тренировки и 10,000 для проверки. Классы являются взаимоисключающими и между ними нет перекрытия.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Нормализуем значения пикселей на величины между 0 и 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Проверка данных

Для проверки корректности данных давайте покажем первые 25 изображений и выведем название классов, соответствующие каждому изображению.


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # Метки CIFAR являются массивами, 
    # поэтому вам нужен дополнительный индекс
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Создание сверточной сети

6 строк кода ниже определяют структуру сети, используя общий паттерн: стек слоев [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) и [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D).

В качестве входа CNN принимает тензор размерности (image_height, image_width, color_channels), игнорируя размер пакета(batch size). Если вам не знакома эта размерность, то в данном случае color_channels это количество цветовых каналов (R,G,B). В этом примере вам нужно сконфигурировать CNN для обработки тензора размерностью (32, 32, 3), что является размерностью CIFAR изображений. Вы можете сделать это, передав аргумент `input_shape` в первый слой.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Давайте посмотрим на архитектуру нашей модели.

In [ ]:
model.summary()

Выше вы можете видеть, что выходные данные каждого слоя Conv2D и MaxPooling2D представляют собой трехмерный тензор вида (высота, ширина, каналы). Размеры ширины и высоты имеют тенденцию уменьшаться по мере того, как вы углубляетесь в сеть. Количество выходных каналов для каждого уровня Conv2D определяется первым аргументом слоя (например, layers.Conv2D(32, (3, 3)) - 32 или layers.Conv2D(64, (3, 3)) - 64). Как правило, по мере уменьшения ширины и высоты вы можете добавить больше выходных каналов на каждом уровне Conv2D.

### Добавляем полносвязные(Dense) слои

Чтобы завершить нашу модель, вы передадите последний выходной тензор из CNN (conv2d_2 (Conv2D) (None, 4, 4, 64)) в один или несколько Dense слоев для выполнения классификации. Dense слои принимают 1D(одномерные) векторы в качестве входных данных, в то время как текущий выход представляет собой трехмерный. Поэтому сначала нужно развернуть трехмерный вектор в одномерный с помощиь слоя Flatten, а затем добавить один или несколько Dense слоев. CIFAR имеет 10 классов изображений, поэтому последний Dense слой имеет 10 выходов, соответствующих количеству классов и активацию softmax, которая применяется для классификации мультиклассовых(больше 2-х классов) данных.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

Полная архитектура модели.

In [ ]:
model.summary()

Как вы видите, выход (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

### Компиляция и тренировка модели

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Оценка модели

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

Наша простая CNN достигла точности тестирования более 70%. Неплохо для нескольких строк кода! Для построения более сложных CNN смотрите примеры использования Keras subclassing API и `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).